In [127]:
import numpy as np
import pandas as pd

import seaborn as sns

In [88]:
exam_info_df = pd.read_csv('data/raw/365_exam_info.csv')
quiz_info_df = pd.read_csv('data/raw/365_quiz_info.csv')
student_engagement_df = pd.read_csv('data/raw/365_student_engagement.csv')
student_hub_questions_df = pd.read_csv('data/raw/365_student_hub_questions.csv')
student_info_df = pd.read_csv('data/raw/365_student_info.csv')
student_learning_df = pd.read_csv('data/raw/365_student_learning.csv')
student_purchases_df = pd.read_csv('data/raw/365_student_purchases.csv')

In [89]:
student_engagement_df.date_engaged = pd.to_datetime(student_engagement_df.date_engaged)
student_hub_questions_df.date_question_asked = pd.to_datetime(student_hub_questions_df.date_question_asked)
student_info_df.date_registered = pd.to_datetime(student_info_df.date_registered)
student_learning_df.date_watched = pd.to_datetime(student_learning_df.date_watched)
student_purchases_df.date_purchased = pd.to_datetime(student_purchases_df.date_purchased)

In [90]:
def has_student_engaged_with_quizzes(student_id: str):
    return (student_engagement_df[student_engagement_df.student_id == student_id].engagement_quizzes == 1).any()

def has_student_engaged_with_exams(student_id: str):
    return (student_engagement_df[student_engagement_df.student_id == student_id].engagement_exams == 1).any()

In [192]:
df = student_info_df.copy()

sum_minutes_watched = pd.merge(student_info_df.student_id, student_learning_df[['student_id', 'minutes_watched']], on='student_id', how='left')
sum_minutes_watched = sum_minutes_watched.fillna(0)
sum_minutes_watched = sum_minutes_watched.groupby('student_id').sum().reset_index()

days_engaged = student_engagement_df[['student_id', 'date_engaged']].groupby('student_id').size().reset_index()
days_engaged = pd.merge(student_info_df.student_id, days_engaged, on='student_id', how='left').fillna(0).astype(np.int32)
days_engaged = days_engaged.rename(columns={0: 'days_engaged'})

df = pd.merge(df.student_id, sum_minutes_watched, on='student_id')
df = pd.merge(df, days_engaged, on='student_id', how='left')

df['engaged_with_quizzes'] = df.student_id.map(has_student_engaged_with_quizzes)
df['engaged_with_exams'] = df.student_id.map(has_student_engaged_with_exams)
df['engaged_with_qa'] = df.student_id.isin(student_hub_questions_df.student_id)
df['subscribed'] = df.student_id.isin(student_purchases_df.student_id)

In [211]:
df[~((df.subscribed == True) & (df.days_engaged == 0))]

,student_id,minutes_watched,days_engaged,engaged_with_quizzes,engaged_with_exams,engaged_with_qa,subscribed
0,258798,0.3,1,False,False,False,False
1,258799,0.0,0,False,False,False,False
2,258800,531.2,29,True,True,False,True
3,258801,0.0,0,False,False,False,False
4,258802,0.0,0,False,False,False,False
...,...,...,...,...,...,...,...
35225,295511,7.8,1,True,False,False,False
35226,295512,10.4,1,True,False,False,False
35227,295513,0.1,1,False,False,False,False
35228,295514,0.0,0,False,False,False,False


In [194]:
df.subscribed.value_counts()

subscribed
False    33095
True      2135
Name: count, dtype: int64

In [209]:
student_purchases_df[student_purchases_df.student_id == 292609]

,purchase_id,student_id,purchase_type,date_purchased
2833,22736,292609,Annual,2022-09-30
